# 003.01c Features - Shift Data to Historical Records

In [1]:
import pathlib
import sys

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.mlab as mlab

%matplotlib inline

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
#add the 'src' directory to path to import modules
PROJECT_DIR = pathlib.Path.cwd().resolve().parent
sys.path.append(str(PROJECT_DIR))

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

DATA_DIR = PROJECT_DIR / 'data'
SCOPED_DATA_DIR = DATA_DIR / '03-scoped'
PROCESSED_DATA_DIR = DATA_DIR / '04-processed'

In [2]:
league = 'english-premier-league'
load_fp = SCOPED_DATA_DIR / league / str(league + '-scoped-1.csv')
seasons = ['2009-2010']


In [3]:
df_orig = pd.read_csv(load_fp, parse_dates = ['date'], index_col=None)
df_orig = df_orig[df_orig['season'].isin(seasons)]
# Calculate Results column
conditions = [df_orig['h_ftgoals'] > df_orig['a_ftgoals'],
              df_orig['h_ftgoals'] == df_orig['a_ftgoals'],
              df_orig['h_ftgoals'] < df_orig['a_ftgoals']]
choices = ['hwin', 'draw', 'awin']
df_orig['result'] = np.select(conditions, choices, default='not-played')
df_orig.rename(columns={'odds_hwin_bbmean': 'odds_hwin',
                      'odds_draw_bbmean': 'odds_draw',
                      'odds_awin_bbmean': 'odds_awin'}, inplace=True)
keeper_cols = ['season', 'date', 'h', 'a','h_shots', 'a_shots', 'result']#,
#               'odds_hwin', 'odds_draw', 'odds_awin']
df_orig = df_orig[keeper_cols]
df_orig.head()

,season,date,h,a,h_shots,a_shots,result
0,2009-2010,2009-08-15,aston-villa,wigan-athletic,11.0,14.0,awin
1,2009-2010,2009-08-15,blackburn-rovers,manchester-city,17.0,8.0,awin
2,2009-2010,2009-08-15,bolton-wanderers,sunderland,11.0,20.0,awin
3,2009-2010,2009-08-15,chelsea,hull-city,26.0,7.0,hwin
4,2009-2010,2009-08-15,everton,arsenal,8.0,15.0,awin


In [4]:
df_bc = df_orig.copy(deep=True)
df_bc.head()

,season,date,h,a,h_shots,a_shots,result
0,2009-2010,2009-08-15,aston-villa,wigan-athletic,11.0,14.0,awin
1,2009-2010,2009-08-15,blackburn-rovers,manchester-city,17.0,8.0,awin
2,2009-2010,2009-08-15,bolton-wanderers,sunderland,11.0,20.0,awin
3,2009-2010,2009-08-15,chelsea,hull-city,26.0,7.0,hwin
4,2009-2010,2009-08-15,everton,arsenal,8.0,15.0,awin


### Putting Code Together

In [5]:
# Form hhaa Records DataFrame
teams = df_bc['h'].unique()

# Get Dataframe by team game home and away
home_team_home_games = []
away_team_away_games = []
for team in teams:
    team_home_game = df_bc[df_bc['h'] == team]
    home_team_home_games.append(team_home_game)
    
    team_away_game = df_bc[df_bc['a'] == team]
    away_team_away_games.append(team_away_game)

# Form historical record dataframes for each team
lags = range(1,len(home_team_home_games[0]))

lagged_home_team_home_games = []
lagged_away_team_away_games = []

feature_cols = [col for col in df_bc.columns if col[0:2] == 'h_']
for home_team_home_game in home_team_home_games:
    lagged_df = home_team_home_game.assign(**{f'h_{col}_-{n}': home_team_home_game[col].shift(n) \
                                              for n in lags \
                                              for col in feature_cols})
    lagged_home_team_home_games.append(lagged_df)

feature_cols = [col for col in df_bc.columns if col[0:2] == 'a_']
for away_team_away_game in away_team_away_games:
    lagged_df = away_team_away_game.assign(**{f'a_{col}_-{n}': away_team_away_game[col].shift(n) \
                                              for n in lags \
                                              for col in feature_cols})
    lagged_away_team_away_games.append(lagged_df)
    
# Join DataFrames together
hh_games = pd.concat([*lagged_home_team_home_games], axis=0, join='inner', sort=True)
hh_games.sort_index(inplace=True)

aa_games = pd.concat([*lagged_away_team_away_games], axis=0, join='inner', sort=True)
aa_games.sort_index(inplace=True)

hhaa_games = pd.concat([hh_games, aa_games], axis=1, join='inner', sort=True)
hhaa_games.sort_index(inplace=True)
#Drop duplicate columns such as h, a etc
hhaa_games = hhaa_games.loc[:,~hhaa_games.columns.duplicated()]

# Re-arrange column sequnce to more intuitive
for col in reversed(['season', 'date', 'h', 'a', 'h_shots', 'a_shots']):
    ser = hhaa_games[col]
    hhaa_games.drop(labels= col, axis=1,inplace = True)
    hhaa_games.insert(0, col, ser)

hhaa_games.head(10)

,season,date,h,a,h_shots,a_shots,h_h_shots_-1,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,result,a_a_shots_-1,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shots_-17,a_a_shots_-18,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9
0,2009-2010,2009-08-15,aston-villa,wigan-athletic,11.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-2010,2009-08-15,blackburn-rovers,manchester-city,17.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-2010,2009-08-15,bolton-wanderers,sunderland,11.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-2010,2009-08-15,chelsea,hull-city,26.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hwin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-2010,2009-08-15,everton,arsenal,8.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2009-2010,2009-08-15,portsmouth,fulham,16.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2009-2010,2009-08-15,stoke-city,burnley,12.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hwin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2009-2010,2009-08-15,wolverhampton-wanderers,west-ham-united,19.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2009-2010,2009-08-16,manchester-united,birmingham-city,26.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hwin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2009-2010,2009-08-16,tottenham-hotspur,liverpool,17.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hwin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
hhaa_games.tail(25)

,season,date,h,a,h_shots,a_shots,h_h_shots_-1,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,result,a_a_shots_-1,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shots_-17,a_a_shots_-18,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9
355,2009-2010,2010-04-25,burnley,liverpool,12.0,12.0,14.0,10.0,11.0,11.0,12.0,15.0,10.0,8.0,8.0,NaN,7.0,22.0,10.0,14.0,8.0,5.0,15.0,8.0,awin,16.0,15.0,9.0,6.0,15.0,12.0,19.0,26.0,6.0,NaN,4.0,11.0,6.0,9.0,6.0,4.0,12.0,7.0
356,2009-2010,2010-04-25,chelsea,stoke-city,29.0,3.0,18.0,25.0,23.0,12.0,26.0,12.0,25.0,27.0,26.0,NaN,13.0,22.0,23.0,9.0,25.0,23.0,18.0,21.0,hwin,9.0,7.0,7.0,8.0,8.0,10.0,13.0,6.0,NaN,NaN,12.0,8.0,14.0,10.0,4.0,7.0,13.0,5.0
357,2009-2010,2010-04-25,everton,fulham,15.0,8.0,10.0,15.0,12.0,9.0,13.0,24.0,18.0,26.0,8.0,NaN,18.0,18.0,11.0,7.0,11.0,13.0,24.0,19.0,hwin,2.0,12.0,12.0,7.0,11.0,10.0,3.0,6.0,9.0,NaN,12.0,8.0,3.0,7.0,9.0,14.0,14.0,3.0
358,2009-2010,2010-05-01,manchester-city,aston-villa,17.0,10.0,8.0,11.0,13.0,17.0,13.0,20.0,20.0,10.0,16.0,NaN,17.0,10.0,12.0,4.0,8.0,13.0,12.0,12.0,hwin,11.0,11.0,8.0,10.0,13.0,13.0,12.0,10.0,12.0,7.0,12.0,13.0,7.0,18.0,10.0,10.0,9.0,7.0
359,2009-2010,2010-05-01,stoke-city,everton,12.0,11.0,11.0,12.0,14.0,9.0,10.0,11.0,2.0,4.0,13.0,12.0,9.0,8.0,13.0,4.0,8.0,15.0,6.0,11.0,draw,11.0,12.0,10.0,5.0,11.0,9.0,17.0,12.0,6.0,17.0,13.0,22.0,10.0,13.0,14.0,6.0,10.0,9.0
360,2009-2010,2010-05-01,birmingham-city,burnley,13.0,12.0,10.0,13.0,16.0,3.0,7.0,19.0,12.0,11.0,8.0,9.0,8.0,7.0,7.0,10.0,15.0,12.0,6.0,11.0,hwin,9.0,10.0,14.0,16.0,6.0,9.0,10.0,6.0,3.0,9.0,9.0,13.0,8.0,13.0,10.0,10.0,11.0,12.0
361,2009-2010,2010-05-01,tottenham-hotspur,bolton-wanderers,25.0,11.0,20.0,19.0,18.0,28.0,10.0,22.0,18.0,11.0,20.0,17.0,13.0,16.0,14.0,13.0,28.0,13.0,25.0,21.0,hwin,13.0,7.0,11.0,17.0,4.0,13.0,15.0,10.0,14.0,20.0,7.0,6.0,7.0,15.0,10.0,8.0,11.0,4.0
362,2009-2010,2010-05-01,portsmouth,wolverhampton-wanderers,12.0,18.0,16.0,14.0,9.0,20.0,21.0,23.0,16.0,21.0,11.0,16.0,6.0,10.0,18.0,11.0,15.0,12.0,14.0,10.0,hwin,5.0,4.0,3.0,4.0,11.0,12.0,15.0,9.0,10.0,9.0,3.0,5.0,7.0,7.0,6.0,12.0,13.0,6.0
363,2009-2010,2010-05-02,fulham,west-ham-united,5.0,7.0,11.0,16.0,12.0,8.0,10.0,17.0,18.0,14.0,4.0,NaN,7.0,10.0,11.0,16.0,14.0,15.0,15.0,10.0,hwin,6.0,9.0,14.0,16.0,15.0,12.0,12.0,12.0,11.0,16.0,10.0,8.0,4.0,10.0,20.0,12.0,8.0,3.0
364,2009-2010,2010-05-02,liverpool,chelsea,7.0,15.0,17.0,12.0,6.0,10.0,27.0,11.0,22.0,27.0,21.0,18.0,18.0,25.0,23.0,12.0,9.0,13.0,13.0,14.0,awin,9.0,20.0,10.0,7.0,27.0,21.0,14.0,18.0,12.0,20.0,11.0,20.0,14.0,7.0,14.0,21.0,16.0,27.0


In [7]:
# Form hhaa Records DataFrame
teams = hhaa_games['h'].unique()

# Get Dataframe by team game home and away
team_games = []
#away_team_away_games = []
for team in teams:
    crit1 = hhaa_games['h'] == team
    crit2 = hhaa_games['a'] == team
    team_game = hhaa_games[crit1 | crit2]
    team_games.append(team_game)
    
team_games[0].head(15)

,season,date,h,a,h_shots,a_shots,h_h_shots_-1,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,result,a_a_shots_-1,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shots_-17,a_a_shots_-18,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9
0,2009-2010,2009-08-15,aston-villa,wigan-athletic,11.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2009-2010,2009-08-24,liverpool,aston-villa,21.0,7.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,2009-2010,2009-08-30,aston-villa,fulham,9.0,6.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hwin,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,2009-2010,2009-09-13,birmingham-city,aston-villa,11.0,12.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,2009-2010,2009-09-19,aston-villa,portsmouth,7.0,17.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hwin,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,2009-2010,2009-09-26,blackburn-rovers,aston-villa,15.0,10.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,hwin,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,2009-2010,2009-10-05,aston-villa,manchester-city,12.0,9.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,draw,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
78,2009-2010,2009-10-17,aston-villa,chelsea,7.0,21.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,9.0,11.0,NaN,NaN,NaN,NaN,NaN,hwin,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,12.0,20.0,NaN,NaN,NaN,NaN,NaN
90,2009-2010,2009-10-24,wolverhampton-wanderers,aston-villa,8.0,12.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,24.0,19.0,NaN,NaN,NaN,NaN,NaN,draw,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN
99,2009-2010,2009-10-31,everton,aston-villa,9.0,13.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,18.0,26.0,8.0,NaN,NaN,NaN,NaN,draw,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,12.0,7.0,NaN,NaN,NaN,NaN,NaN


In [8]:
# Get the indices for the team in question
h_indices = team_games[0][team_games[0]['h'] == 'aston-villa'].index

a_indices = team_games[0][team_games[0]['a'] == 'aston-villa'].index
print(h_indices)
print(a_indices)

Int64Index([0, 33, 47, 75, 78, 107, 129, 139, 168, 192, 206, 216, 247, 259, 298, 303, 334, 354, 374], dtype='int64')
Int64Index([25, 44, 57, 90, 99, 106, 123, 154, 162, 184, 222, 239, 286, 291, 309, 317, 344, 347, 358], dtype='int64')


In [9]:
# we want to get the home team away record - designated h_a
# This means we need to get the record based on the previos time the team played away
# The indexes we want to add columns to are h_indices
print(h_indices)
# The indexes of the times they played away are in a_indices
# We want to form h_a, so we want to shift the a_indices data down to match the h indices
# We want tyhe closest lower indices
shifted_indices = [a_indices[a_indices < number].max() for number in h_indices]
print(shifted_indices)
# We now have
# team_games[0].loc[h_indices] = team_games[0].loc[shifted_indices][data]
# We will form the data
# We want to get the away records so we will grab anything that has got an a_a at the start
data_cols = [col for col in team_games[0].columns if col[0:2] == 'a_']
print(data_cols)
# the column labeled a_shots is actually the previous away game, so we need to increment all the columns by 1
numbered_cols = [col.rsplit('_',1)[0] + '_' + str(int(col.split('_')[-1])-1) for col in data_cols if not col.split('_')[-1].isalpha()]
print(numbered_cols)
# Now deal with the missing -1 value
new_minus_one = 'a_' + data_cols[0] + '_-1'
print(new_minus_one)
# print(data_cols[0])
numbered_cols.insert(0, new_minus_one)
print(numbered_cols)
# Now we are getting h_a data, so we need to relabel the first a as a h
renamed_cols = ['h' + col[1:] for col in numbered_cols]
print(renamed_cols)
all_cols = list(team_games[0].columns) + renamed_cols
print(all_cols)


Int64Index([0, 33, 47, 75, 78, 107, 129, 139, 168, 192, 206, 216, 247, 259, 298, 303, 334, 354, 374], dtype='int64')
[nan, 25, 44, 57, 57, 106, 123, 123, 162, 184, 184, 184, 239, 239, 291, 291, 317, 347, 358]
['a_shots', 'a_a_shots_-1', 'a_a_shots_-10', 'a_a_shots_-11', 'a_a_shots_-12', 'a_a_shots_-13', 'a_a_shots_-14', 'a_a_shots_-15', 'a_a_shots_-16', 'a_a_shots_-17', 'a_a_shots_-18', 'a_a_shots_-2', 'a_a_shots_-3', 'a_a_shots_-4', 'a_a_shots_-5', 'a_a_shots_-6', 'a_a_shots_-7', 'a_a_shots_-8', 'a_a_shots_-9']
['a_a_shots_-2', 'a_a_shots_-11', 'a_a_shots_-12', 'a_a_shots_-13', 'a_a_shots_-14', 'a_a_shots_-15', 'a_a_shots_-16', 'a_a_shots_-17', 'a_a_shots_-18', 'a_a_shots_-19', 'a_a_shots_-3', 'a_a_shots_-4', 'a_a_shots_-5', 'a_a_shots_-6', 'a_a_shots_-7', 'a_a_shots_-8', 'a_a_shots_-9', 'a_a_shots_-10']
a_a_shots_-1
['a_a_shots_-1', 'a_a_shots_-2', 'a_a_shots_-11', 'a_a_shots_-12', 'a_a_shots_-13', 'a_a_shots_-14', 'a_a_shots_-15', 'a_a_shots_-16', 'a_a_shots_-17', 'a_a_shots_-18', '

In [10]:
# So now we can have a go at joining the data
team_games[0] = team_games[0].reindex(all_cols, axis=1)
#team_games[0].head()
team_games[0].loc[h_indices, renamed_cols] = team_games[0].loc[shifted_indices, data_cols].values
team_games[0].head(15)
#team_games[0].loc[shifted_indices, data_cols]

/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,season,date,h,a,h_shots,a_shots,h_h_shots_-1,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,result,a_a_shots_-1,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shots_-17,a_a_shots_-18,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9,h_a_shots_-1,h_a_shots_-2,h_a_shots_-11,h_a_shots_-12,h_a_shots_-13,h_a_shots_-14,h_a_shots_-15,h_a_shots_-16,h_a_shots_-17,h_a_shots_-18,h_a_shots_-19,h_a_shots_-3,h_a_shots_-4,h_a_shots_-5,h_a_shots_-6,h_a_shots_-7,h_a_shots_-8,h_a_shots_-9,h_a_shots_-10
0,2009-2010,2009-08-15,aston-villa,wigan-athletic,11.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2009-2010,2009-08-24,liverpool,aston-villa,21.0,7.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,2009-2010,2009-08-30,aston-villa,fulham,9.0,6.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hwin,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,2009-2010,2009-09-13,birmingham-city,aston-villa,11.0,12.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,2009-2010,2009-09-19,aston-villa,portsmouth,7.0,17.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hwin,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,2009-2010,2009-09-26,blackburn-rovers,aston-villa,15.0,10.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,hwin,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,2009-2010,2009-10-05,aston-villa,manchester-city,12.0,9.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,draw,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,2009-2010,2009-10-17,aston-villa,chelsea,7.0,21.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,9.0,11.0,NaN,NaN,NaN,NaN,NaN,hwin,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,12.0,20.0,NaN,NaN,NaN,NaN,NaN,10.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,2009-2010,2009-10-24,wolverhampton-wanderers,aston-villa,8.0,12.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,24.0,19.0,NaN,NaN,NaN,NaN,NaN,draw,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,2009-2010,2009-10-31,everton,aston-villa,9.0,13.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,18.0,26.0,8.0,NaN,NaN,NaN,NaN,draw,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,12.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# we want to get the away team home record - designated a_h
# This means we need to get the record based on the previous time the team played at home
# The indexes we want to add columns to are a_indices
print(a_indices)
# The indexes of the times they played away are in a_indices
# We want to form h_a, so we want to shift the a_indices data down to match the h indices
# We want tyhe closest lower indices
shifted_indices = [h_indices[h_indices < number].max() for number in a_indices]
print(shifted_indices)
# We now have
# team_games[0].loc[h_indices] = team_games[0].loc[shifted_indices][data]
# We will form the data
# We want to get the home records so we will grab anything that has got an a at the start
data_cols = [col for col in team_games[0].columns if col[0:3] == 'h']
print(data_cols)
# the column labeled a_shots is actually the previous home game, so we need to increment all the columns by 1
numbered_cols = [col.rsplit('_',1)[0] + '_' + str(int(col.split('_')[-1])-1) for col in data_cols if not col.split('_')[-1].isalpha()]
print(numbered_cols)
# Now deal with the missing -1 value
new_minus_one = 'h_' + data_cols[0] + '_-1'
print(new_minus_one)
# print(data_cols[0])
numbered_cols.insert(0, new_minus_one)
print(numbered_cols)
# Now we are getting h_a data, so we need to relabel the first a as a h
renamed_cols = ['a' + col[1:] for col in numbered_cols]
print(renamed_cols)
all_cols = list(team_games[0].columns) + renamed_cols
print('\n')
print(all_cols)

Int64Index([25, 44, 57, 90, 99, 106, 123, 154, 162, 184, 222, 239, 286, 291, 309, 317, 344, 347, 358], dtype='int64')
[0, 33, 47, 78, 78, 78, 107, 139, 139, 168, 216, 216, 259, 259, 303, 303, 334, 334, 354]
['h']
[]
h_h_-1
['h_h_-1']
['a_h_-1']


['season', 'date', 'h', 'a', 'h_shots', 'a_shots', 'h_h_shots_-1', 'h_h_shots_-10', 'h_h_shots_-11', 'h_h_shots_-12', 'h_h_shots_-13', 'h_h_shots_-14', 'h_h_shots_-15', 'h_h_shots_-16', 'h_h_shots_-17', 'h_h_shots_-18', 'h_h_shots_-2', 'h_h_shots_-3', 'h_h_shots_-4', 'h_h_shots_-5', 'h_h_shots_-6', 'h_h_shots_-7', 'h_h_shots_-8', 'h_h_shots_-9', 'result', 'a_a_shots_-1', 'a_a_shots_-10', 'a_a_shots_-11', 'a_a_shots_-12', 'a_a_shots_-13', 'a_a_shots_-14', 'a_a_shots_-15', 'a_a_shots_-16', 'a_a_shots_-17', 'a_a_shots_-18', 'a_a_shots_-2', 'a_a_shots_-3', 'a_a_shots_-4', 'a_a_shots_-5', 'a_a_shots_-6', 'a_a_shots_-7', 'a_a_shots_-8', 'a_a_shots_-9', 'h_a_shots_-1', 'h_a_shots_-2', 'h_a_shots_-11', 'h_a_shots_-12', 'h_a_shots_-13', 'h_a_shots_-14'

In [12]:
all_cols

['season',
 'date',
 'h',
 'a',
 'h_shots',
 'a_shots',
 'h_h_shots_-1',
 'h_h_shots_-10',
 'h_h_shots_-11',
 'h_h_shots_-12',
 'h_h_shots_-13',
 'h_h_shots_-14',
 'h_h_shots_-15',
 'h_h_shots_-16',
 'h_h_shots_-17',
 'h_h_shots_-18',
 'h_h_shots_-2',
 'h_h_shots_-3',
 'h_h_shots_-4',
 'h_h_shots_-5',
 'h_h_shots_-6',
 'h_h_shots_-7',
 'h_h_shots_-8',
 'h_h_shots_-9',
 'result',
 'a_a_shots_-1',
 'a_a_shots_-10',
 'a_a_shots_-11',
 'a_a_shots_-12',
 'a_a_shots_-13',
 'a_a_shots_-14',
 'a_a_shots_-15',
 'a_a_shots_-16',
 'a_a_shots_-17',
 'a_a_shots_-18',
 'a_a_shots_-2',
 'a_a_shots_-3',
 'a_a_shots_-4',
 'a_a_shots_-5',
 'a_a_shots_-6',
 'a_a_shots_-7',
 'a_a_shots_-8',
 'a_a_shots_-9',
 'h_a_shots_-1',
 'h_a_shots_-2',
 'h_a_shots_-11',
 'h_a_shots_-12',
 'h_a_shots_-13',
 'h_a_shots_-14',
 'h_a_shots_-15',
 'h_a_shots_-16',
 'h_a_shots_-17',
 'h_a_shots_-18',
 'h_a_shots_-19',
 'h_a_shots_-3',
 'h_a_shots_-4',
 'h_a_shots_-5',
 'h_a_shots_-6',
 'h_a_shots_-7',
 'h_a_shots_-8',
 'h_a

In [13]:
stop

NameError: name 'stop' is not defined

## Functions to Cut Data to Defined Shape prior to PreProcessing

In [ ]:
import re

def get_n_past_games(df_orig, past_games=2, dropna_rows=True):
    """
    Restrict historical records to the last n games
    If n = 2, then will get
    home team
        last 2 games played at home ie. h_h_feature-1, h_h_feature-2
        last 2 games played away ie h_a_feature-1, h_a_feature-2
    away team
        last 2 games played at home ie a_h_feature-1, a_h_feature-2
        last 2 games played away ie a_a_feature-1, a_a_feature-2
    where -1 is the most recent game prior to the current game, and -2 is the game before that
    the current games result is in result
    """
    df = df_orig.copy(deep=True)
    keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(hist_games+1) \
               or col.isalpha() \
               or 'odds' in col]
    
    # Rename the target columns to start with prefix 'res_'
    res_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
                and int(col.split('_')[-1]) == 0]
    res_cols.append('result')
    res_cols = [col.replace('0', 'res') for col in res_cols]
    print(res_cols)
    print(res_cols)
    df = df[keeper_cols]
    return df
    
df_cut = get_n_past_games(df_load)
# # Restrict Historical records to last 4 games per team
# df_cut = df_load.copy(deep=True)
# hist_games = 2
# keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
#                and int(col.split('_')[-1]) > -(hist_games+1) \
#                and int(col.split('_')[-1]) < 0 \
#                or col.isalpha() \
#                or 'odds' in col]
# df_cut = df_cut[keeper_cols]
df_cut.head()

In [ ]:
import re

def get_n_past_games(df_orig, past_games=2, dropna_rows=True):
    """
    Restrict historical records to the last n games
    If n = 2, then will get
    home team
        last 2 games played at home ie. h_h_feature-1, h_h_feature-2
        last 2 games played away ie h_a_feature-1, h_a_feature-2
    away team
        last 2 games played at home ie a_h_feature-1, a_h_feature-2
        last 2 games played away ie a_a_feature-1, a_a_feature-2
    where -1 is the most recent game prior to the current game, and -2 is the game before that
    the current games result is in result
    """
    df = df_orig.copy(deep=True)
    keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(hist_games+1) \
               and int(col.split('_')[-1]) < 0 \
               or col.isalpha() \
               or 'odds' in col]
df_cut = df_cut[keeper_cols]
    

# Restrict Historical records to last 4 games per team
df_cut = df_load.copy(deep=True)
hist_games = 2
keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(hist_games+1) \
               and int(col.split('_')[-1]) < 0 \
               or col.isalpha() \
               or 'odds' in col]
df_cut = df_cut[keeper_cols]
df_cut.head(10)